In [1]:
import ast # For generating the abstract syntax tree
import dis # For generatiog the intermediate representation
from IPython.display import display, Markdown # For better displayin gin a jupyter notebook
from astroid import parse


### Loading the program

In [2]:
with open('test-programs/simple_quantum.py', 'r') as file:
    code = file.read()
code

'from qiskit import QuantumCircuit\n\ndef quantum_rotation_program(angles):\n    qc = QuantumCircuit(1, 1)\n    \n    for angle in angles:\n        qc.rx(angle, 0)\n        qc.ry(angle * 2, 0)\n    \n    qc.measure(0, 0)\n    return qc\n\n# Example usage\nangles = [0.1, 0.2, 0.3]\ncircuit = quantum_rotation_program(angles)'

### AST with ast module

In [6]:
tree = ast.parse(code)

formatted_ast = ast.dump(tree, indent=4)

display(Markdown(f"```python\n{formatted_ast}\n```"))

```python
Module(
    body=[
        ImportFrom(
            module='qiskit',
            names=[
                alias(name='QuantumCircuit')],
            level=0),
        FunctionDef(
            name='quantum_rotation_program',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='angles')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='qc', ctx=Store())],
                    value=Call(
                        func=Name(id='QuantumCircuit', ctx=Load()),
                        args=[
                            Constant(value=1),
                            Constant(value=1)],
                        keywords=[])),
                For(
                    target=Name(id='angle', ctx=Store()),
                    iter=Name(id='angles', ctx=Load()),
                    body=[
                        Expr(
                            value=Call(
                                func=Attribute(
                                    value=Name(id='qc', ctx=Load()),
                                    attr='rx',
                                    ctx=Load()),
                                args=[
                                    Name(id='angle', ctx=Load()),
                                    Constant(value=0)],
                                keywords=[])),
                        Expr(
                            value=Call(
                                func=Attribute(
                                    value=Name(id='qc', ctx=Load()),
                                    attr='ry',
                                    ctx=Load()),
                                args=[
                                    BinOp(
                                        left=Name(id='angle', ctx=Load()),
                                        op=Mult(),
                                        right=Constant(value=2)),
                                    Constant(value=0)],
                                keywords=[]))],
                    orelse=[]),
                Expr(
                    value=Call(
                        func=Attribute(
                            value=Name(id='qc', ctx=Load()),
                            attr='measure',
                            ctx=Load()),
                        args=[
                            Constant(value=0),
                            Constant(value=0)],
                        keywords=[])),
                Return(
                    value=Name(id='qc', ctx=Load()))],
            decorator_list=[]),
        Assign(
            targets=[
                Name(id='angles', ctx=Store())],
            value=List(
                elts=[
                    Constant(value=0.1),
                    Constant(value=0.2),
                    Constant(value=0.3)],
                ctx=Load())),
        Assign(
            targets=[
                Name(id='circuit', ctx=Store())],
            value=Call(
                func=Name(id='quantum_rotation_program', ctx=Load()),
                args=[
                    Name(id='angles', ctx=Load())],
                keywords=[]))],
    type_ignores=[])
```

In [7]:
module = parse(code)
module

<Module l.0 at 0x21a40b63f70>

### IR with dis module

In [9]:
compiled_code = compile(code, filename="simple_loop.py", mode="exec")

IR = dis.dis(compiled_code)

  1           0 LOAD_CONST               0 (0)
              2 LOAD_CONST               1 (('QuantumCircuit',))
              4 IMPORT_NAME              0 (qiskit)
              6 IMPORT_FROM              1 (QuantumCircuit)
              8 STORE_NAME               1 (QuantumCircuit)
             10 POP_TOP

  3          12 LOAD_CONST               2 (<code object quantum_rotation_program at 0x00000218AE415A50, file "simple_loop.py", line 3>)
             14 LOAD_CONST               3 ('quantum_rotation_program')
             16 MAKE_FUNCTION            0
             18 STORE_NAME               2 (quantum_rotation_program)

 14          20 BUILD_LIST               0
             22 LOAD_CONST               4 ((0.1, 0.2, 0.3))
             24 LIST_EXTEND              1
             26 STORE_NAME               3 (angles)

 15          28 LOAD_NAME                2 (quantum_rotation_program)
             30 LOAD_NAME                3 (angles)
             32 CALL_FUNCTION            1
   

### IR with LLVM

In [9]:
from numba import jit
import llvmlite.binding as llvm

@jit
def example_func(n):
    total = 0
    for i in range(n):
        total += i * 2
    return total

example_func(10)

# Print LLVM IR
print(example_func.inspect_llvm())

{(int64,): '; ModuleID = \'example_func\'\nsource_filename = "<string>"\ntarget datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"\ntarget triple = "x86_64-pc-windows-msvc"\n\n@.const.example_func = internal constant [13 x i8] c"example_func\\00"\n@_ZN08NumbaEnv8__main__12example_funcB2v1B38c8tJTIeFIjxB2IKSgI4CrvQClQZ6FczSBAA_3dEx = common local_unnamed_addr global i8* null\n@".const.missing Environment: _ZN08NumbaEnv8__main__12example_funcB2v1B38c8tJTIeFIjxB2IKSgI4CrvQClQZ6FczSBAA_3dEx" = internal constant [105 x i8] c"missing Environment: _ZN08NumbaEnv8__main__12example_funcB2v1B38c8tJTIeFIjxB2IKSgI4CrvQClQZ6FczSBAA_3dEx\\00"\n@PyExc_RuntimeError = external global i8\n\n; Function Attrs: nofree norecurse nosync nounwind writeonly\ndefine i32 @_ZN8__main__12example_funcB2v1B38c8tJTIeFIjxB2IKSgI4CrvQClQZ6FczSBAA_3dEx(i64* noalias nocapture writeonly %retptr, { i8*, i32, i8*, i8*, i32 }** noalias nocapture readnone %excinfo, i64 %arg.n) local_unnamed_a